In [ ]:
!pip install geopy requests numpy pandas torch nltk scikit-learn scipy

import requests
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.ensemble import IsolationForest
from scipy.spatial.distance import pdist, squareform
from itertools import permutations

# Load NLTK VADER for Sentiment Analysis
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Step 1: Get User Location (Zip Code -> Lat/Lon)
def get_coordinates_from_zip(zip_code):
    geolocator = Nominatim(user_agent="business_crowd_predictor")
    location = geolocator.geocode(f"{zip_code}, USA")
    if location:
        return location.latitude, location.longitude
    return None, None

zip_code = input("Enter your zip code: ")
latitude, longitude = get_coordinates_from_zip(zip_code)
if latitude and longitude:
    CURRENT_LOCATION = (latitude, longitude)

# Step 2: Fetch Business Data
BUSINESS_TYPES = {"restaurant": "amenity=restaurant",
    "cafe": "amenity=cafe",
    "store": "shop",
    "pharmacy": "amenity=pharmacy",
    "supermarket": "shop=supermarket",
    "car_wash": "amenity=car_wash",
    "gym": "leisure=fitness_centre" }

def fetch_business_data(lat, lng, radius, business_key, business_value):
    OVERPASS_URL = "https://overpass-api.de/api/interpreter"
    query = f"""[out:json];(node[{business_value}](around:{radius},{lat},{lng}););out center;"""
    response = requests.get(OVERPASS_URL, params={"data": query})
    if response.status_code != 200:
        return []
    data = response.json()
    return [{"Name": place["tags"].get("name", "Unknown"),
             "Business Type": business_key,
             "Latitude": place.get("lat", place.get("center", {}).get("lat")),
             "Longitude": place.get("lon", place.get("center", {}).get("lon"))}
            for place in data.get("elements", []) if "tags" in place]

all_businesses = []
for business_key, business_value in BUSINESS_TYPES.items():
    businesses = fetch_business_data(CURRENT_LOCATION[0], CURRENT_LOCATION[1], 8000, business_key, business_value)
    all_businesses.extend(businesses)

df = pd.DataFrame(all_businesses)

# Step 3: Simulate Ratings, Reviews & Busy Score
df["Rating"] = np.random.uniform(3.5, 5.0, len(df))
df["User Ratings"] = np.random.randint(5, 500, len(df))
df["Busy Score"] = df["Rating"] * df["User Ratings"]
df["Sample Review"] = np.random.choice([
    "Amazing service!", "Overpriced but decent.", "Worst experience ever.", "Good food, friendly staff.", "Not bad, just okay."
], len(df))
df["Sentiment"] = df["Sample Review"].apply(lambda x: "Positive" if sia.polarity_scores(x)["compound"] >= 0.05 else "Negative")

# Step 4: AI-Based Anomaly Detection
model = IsolationForest(contamination=0.05)
df["Anomaly"] = model.fit_predict(df[["Rating", "User Ratings"]])
df = df[df["Anomaly"] == 1]

avg_busy_scores = df.groupby("Business Type")["Busy Score"].mean().to_dict()

df["Status"] = df.apply(
    lambda row: "Crowded" if row["Busy Score"] > avg_busy_scores[row["Business Type"]] * 1.1
    else "Less Busy", axis=1)

# Step 5: Predict Future Crowds (Markov Chain)
def predict_future_crowd(df):
    """ Simulate future crowd levels at 15 mins, 30 mins, and 60 mins using Markov Chain transition probabilities. """

    transition_matrix = {
        "Less Busy": {"Less Busy": 0.7, "Crowded": 0.3},
        "Crowded": {"Less Busy": 0.4, "Crowded": 0.6}}

    def simulate_status(current_status, steps):
        """Simulates the crowd level after a given number of steps (time intervals)."""
        status = current_status
        for _ in range(steps):
            status = "Crowded" if random.choices(
                ["Less Busy", "Crowded"],
                weights=[transition_matrix[status]["Less Busy"], transition_matrix[status]["Crowded"]]
            )[0] == "Crowded" else "Less Busy"
        return status

    df["Status in 15 mins"] = df["Status"].apply(lambda x: simulate_status(x, 1))
    df["Status in 30 mins"] = df["Status"].apply(lambda x: simulate_status(x, 2))
    df["Status in 60 mins"] = df["Status"].apply(lambda x: simulate_status(x, 4))

    return df

# Apply future crowd prediction
df = predict_future_crowd(df)

# Step 6: Interactive AI Assistant
def interactive_ai():
    print("\nWelcome to the AI Assistant!")
    print("What are your top 3 priorities right now?")
    print("1. Eating (Restaurant, Cafe)")
    print("2. Shopping (Store, Supermarket, Pharmacy)")
    print("3. Fitness (Gym)")
    print("4. Car Services (Car Wash)")

    choices = input("Enter up to 3 numbers separated by commas (e.g., 1,3): ").split(",")
    choices = [x.strip() for x in choices]

    priority_mapping = {
        "1": ["restaurant", "cafe"],
        "2": ["store", "supermarket", "pharmacy"],
        "3": ["gym"],
        "4": ["car_wash"]}

    selected_priorities = [priority_mapping[x] for x in choices if x in priority_mapping]
    selected_priorities = [item for sublist in selected_priorities for item in sublist]

    print("\nWhat is most important to you?")
    print("1. Highest rating")
    print("2. Closest distance")
    print("3. Least crowded")

    preference = input("Enter your preference (1, 2, or 3): ").strip()

    recommendations = df[(df["Business Type"].isin(selected_priorities)) & (df["Status"] == "Less Busy")]

    if preference == "1":
        recommendations = recommendations.sort_values("Rating", ascending=False)
    elif preference == "2":
        recommendations["Distance"] = recommendations.apply(
            lambda row: geodesic((CURRENT_LOCATION[0], CURRENT_LOCATION[1]),
                                 (row["Latitude"], row["Longitude"])).miles, axis=1)
        recommendations = recommendations.sort_values("Distance")
    elif preference == "3":
        recommendations = recommendations.sort_values("Busy Score")

    if recommendations.empty:
        print("\n⚠️ All locations in your selected categories are currently crowded.")
        print("Would you like to wait or try a different category?")
        retry = input("Enter 'wait' or 'new category': ").strip().lower()
        if retry == "wait":
            print("We will check again in 15 minutes!")
        else:
            interactive_ai()

    else:
        print("\nHere are the best places for you right now:")
        display(recommendations.head(5))

# Run AI Assistant
interactive_ai()
